## SQL Project


The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers.

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

Instructions for completing the task:

- Describe the goals of the study.

- Study the tables (print the first rows).

- Make an SQL query for each of the tasks.

- Output the results of each query in the Notebook.

- Describe your conclusions for each of the tasks.

## Goal of the Project

The goals of this study are:

- To analyze the book database to identify trends, patterns, and insights that can inform product development and marketing strategy for a new book-related product. 

- To look at metrics like publication dates, reviews, ratings, publishers, and authors to understand what kinds of books and authors are popular, highly rated, and well reviewed.

- To generate a value proposition and make data-driven decisions about what features and target audience to focus on for a new book product.

## Import libraries & Initialization

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student',         # username
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Querying the data

In [2]:
# SQL queries to retrieve the first rows of each table
query_books = "SELECT * FROM books LIMIT 5;"
query_authors = "SELECT * FROM authors LIMIT 5;"
query_publishers = "SELECT * FROM publishers LIMIT 5;"
query_ratings = "SELECT * FROM ratings LIMIT 5;"
query_reviews = "SELECT * FROM reviews LIMIT 5;"

# Execute the SQL queries and store the results in DataFrames
df_books = pd.read_sql(query_books, engine)
df_authors = pd.read_sql(query_authors, engine)
df_publishers = pd.read_sql(query_publishers, engine)
df_ratings = pd.read_sql(query_ratings, engine)
df_reviews = pd.read_sql(query_reviews, engine)

In [3]:
# Print the first rows of each DataFrame
print("First rows of the 'books' table:")
display(df_books)

print("\nFirst rows of the 'authors' table:")
display(df_authors)

print("\nFirst rows of the 'publishers' table:")
display(df_publishers)

print("\nFirst rows of the 'ratings' table:")
display(df_ratings)

print("\nFirst rows of the 'reviews' table:")
display(df_reviews)

First rows of the 'books' table:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



First rows of the 'authors' table:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



First rows of the 'publishers' table:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



First rows of the 'ratings' table:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



First rows of the 'reviews' table:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Overview Conclusions:

- The database appears to be well-structured, with separate tables for books, authors, publishers, ratings, and reviews.
- There is a clear relationship between the tables, with foreign keys (author_id, book_id, publisher_id) linking the books to their respective authors and publishers.
- Users are actively engaging with the books by providing ratings and text reviews, as seen in the ratings and reviews tables.
- The data allows for analysis of book ratings, author performance, and user feedback to gain insights into popular books and author success.

## Number of books released after January 1, 2000.

In [4]:
# SQL query
books_released = """
SELECT COUNT(*) 
FROM books
WHERE publication_date > '2000-01-01'
"""


# Execute the SQL query and store the result in a DataFrame
result = engine.execute(books_released)
post_2000_count = result.fetchone()[0]
print(post_2000_count)

819


Based on the query result of 819 books released after January 1, 2000, we can make a few conclusions:

- There are a significant number of recently published books in the database - over 800 were published in the last ~20 years. 

- This suggests the book database has substantial coverage of contemporary titles and not just older public domain works.

- Users interested in latest releases and current authors would find a lot of relevant content.

- Compared to the full size of the database (which we don't know yet), this 819 post-2000 count likely represents a meaningful proportion of total books.

- Services and features catering to readers of new and current titles would be applicable for this database. This includes reviews, ratings, recommendations, ability to filter by publication date, etc. 

- Analysis of ratings and reviews could be done on this set of post-2000 books to look at trends for modern books specifically.

In summary, the significant number of recent titles demonstrates this database goes beyond just old classics and has substantial modern content. This informs what types of users, features, and analytics would be relevant as we continue exploring the data. The post-2000 count provides an initial benchmark for gauging the database contents.

## Number of user reviews and the average rating for each book

In [5]:
# Execute the SQL query and store the result in a DataFrame
query = """
SELECT b.book_id, b.title, COUNT(DISTINCT rv.review_id) AS num_reviews, AVG(r.rating) AS average_rating
FROM books b
LEFT JOIN ratings r ON b.book_id = r.book_id
LEFT JOIN reviews rv ON b.book_id = rv.book_id
GROUP BY b.book_id, b.title;
"""

# Execute the SQL query and store the result in a DataFrame
df_reviews_ratings = pd.read_sql(query, engine)

# Display the DataFrame
display(df_reviews_ratings)

,book_id,title,num_reviews,average_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


From the provided data showing the number of reviews and average ratings for each book, we can draw several conclusions:

- Popular Books with High Reviews: Books with a higher number of reviews are generally more popular among readers. Books with a large number of reviews (e.g., "Salem's Lot" with 2 reviews) may have garnered significant attention and interest from readers.

- Rating and Popularity: Books with higher average ratings (e.g., 4.5 and above) tend to be well-received by readers. These books have likely resonated positively with the audience, leading to higher ratings.

- Diverse Reader Preferences: The range of average ratings (e.g., from 2.5 to 4.67) indicates that different books cater to varying reader preferences. Some books might be polarizing, appealing strongly to certain readers while not to others.


## Publisher that has released the greatest number of books with more than 50

This is what we will query:

1. Joining the publishers table to link each book to its publisher - this brings in the publisher name data.

2. The WHERE clause filters to only books with >50 pages. This focuses on full books rather than brochures.

3. Grouping by publisher_id and publisher is crucial to aggregate at the publisher level. 

4. Using COUNT() lets us tally the number of books per publisher.

5. Ordering by the counted books in descending order and limiting to 5 row 

The joins, filtering, grouping, aggregation with COUNT(), and ordering/limiting work together to effectively answer this analysis question.

In [6]:
# SQL query
query = """
SELECT p.publisher_id, p.publisher, COUNT(b.book_id) AS num_books
FROM publishers p
JOIN books b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY num_books DESC
LIMIT 5;
"""

# Execute the SQL query and store the result in a DataFrame
df_publisher_most_books = pd.read_sql(query, engine)

# Display the result
display(df_publisher_most_books)

,publisher_id,publisher,num_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19


From the provided data showing the publishers with the greatest number of books having more than 50 pages, we can draw several conclusions:

- Penguin Books is the Leading Publisher: Among the listed publishers, `Penguin Books` has released the greatest number of books with more than 50 pages, with a total of 42 books meeting the criteria.

- Diverse Publishing Houses: The list includes publishers with varying specialties and focuses, such as `Vintage,` `Grand Central Publishing,` `Penguin Classics`, and `Bantam`. Each publisher has its unique catalog of books meeting the specified criteria.

- Penguin Imprints Dominate: The `Penguin Books` publisher, along with its imprint `Penguin Classics`, contributes significantly to the number of books meeting the criteria. This indicates that Penguin's various imprints have a diverse range of book offerings.

## Author with the highest average book rating (at least 50 ratings)

In this query:

1. We join the `authors` table with the `books` table on the `author_id` column and then join the `books` table with the `ratings` table on the `book_id` column to link `authors`, `books`, and `ratings`.
2. We use a subquery to find the `book_ids` of books that have at least 50 ratings. The subquery groups the ratings by `book_id` and filters books with a count of ratings greater than or equal to 50.
3. We filter the main query to include only the books with `book_ids` obtained from the subquery, i.e., books with at least 50 ratings.
4. We group the results by `author_id` and `author_name` to calculate the average rating for each author's books.
5. Finally, we sort the results in descending order of the average rating and use LIMIT 5 to get the top 5 author with the highest average rating.

In [7]:
# SQL query
query = """
SELECT a.author_id, a.author, AVG(r.rating) AS average_rating
FROM authors a
JOIN books b ON a.author_id = b.author_id
JOIN ratings r ON b.book_id = r.book_id
WHERE b.book_id IN (
    SELECT book_id
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(*) >= 50
)
GROUP BY a.author_id, a.author
ORDER BY average_rating DESC
LIMIT 5;
"""

# Execute the SQL query and store the result in a DataFrame
df_author_highest_avg_rating = pd.read_sql(query, engine)

# Display the result
display(df_author_highest_avg_rating)

,author_id,author,average_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.246914
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645


From the provided data showing the authors with the highest average book ratings (considering only books with at least 50 ratings), we can draw several conclusions:

- Top-Rated Authors: The authors listed have achieved high average ratings for their books, indicating that their works are well-received by readers.

- Diverse Genres: The authors on the list represent a diverse range of genres and writing styles. This suggests that different types of books can achieve high ratings when they resonate with readers.

- Recognizable Names: Many of the authors listed are well-known and have a significant readership. Established authors like J.K. Rowling, Markus Zusak, J.R.R. Tolkien, Louisa May Alcott, and Rick Riordan have a track record of producing popular and highly rated books.


## Average number of text reviews among users who rated more than 50 books.

What we will query:

1. We join the `ratings` table with the `reviews` table on the `book_id` column to link ratings with their corresponding text reviews.
2. We use a subquery to find the usernames of users who have rated more than 50 books. The subquery groups the ratings by username and filters users with a count of distinct `book_ids` greater than 50.
3. We filter the main query to include only the ratings from the users obtained from the subquery, i.e., users who have rated more than 50 books.
4. We group the results by username to calculate the count of distinct text reviews for each user.
5. The final result will include the username and the corresponding `num_text_reviews`.

In [9]:
# SQL query to find the average number of text reviews among users who rated more than 50 books
query = """
SELECT
    AVG(sub.review_count) AS avg_review_count
FROM
    (SELECT username, COUNT(review_id) AS review_count 
     FROM reviews 
     GROUP BY username) AS sub
WHERE
    sub.username IN (SELECT username 
                     FROM ratings 
                     GROUP BY username 
                     HAVING COUNT(rating_id) > 50);
"""

# Execute the SQL query and store the result in a DataFrame
df_avg_text_reviews = pd.read_sql(query, engine)

# Extract the average number of text reviews from the DataFrame using the correct alias
average_text_reviews = df_avg_text_reviews['avg_review_count'].values[0]

# Round the average_text_reviews to 2 decimal points
average_text_reviews = round(average_text_reviews, 2)

# Display the average number of text reviews
print("Average number of text reviews among users who rated more than 50 books:", average_text_reviews)

Average number of text reviews among users who rated more than 50 books: 24.33


Based on the revised average number of text reviews among users who rated more than 50 books (approximately 24.33), we can draw the following new conclusion:

1. **Emerging Reviewers**: The average number of text reviews among users who rated more than 50 books being approximately 24.33 suggests that there is a significant presence of emerging reviewers within the community. These users may be relatively new to the reviewing scene but are actively engaging with a considerable number of books and sharing their thoughts through text reviews.

2. **Growing Community**: The presence of multiple users who have rated more than 50 books and contributed text reviews indicates a growing and vibrant community of readers and reviewers. This growth could foster more diverse discussions and opinions about various books, leading to a richer reading experience for the entire community.

3. **Quality over Quantity**: While the average number of text reviews may not be exceptionally high, it emphasizes the importance of quality reviews over sheer quantity. Users who contribute thoughtful and meaningful reviews enhance the overall reviewing experience for others and play a crucial role in guiding fellow readers in their book choices.

## Overall Overview

This comprehensive SQL analysis of the book database yielded valuable insights to guide product development and marketing for a new book-related product. By querying data on books, authors, publishers, ratings, and reviews, we were able to draw several key findings and conclusions:

**Goal and Approach**

The primary goal of the analysis was to gain insights from the book database to inform the development of a new book-related product. Using SQL queries, we examined various metrics, including publication dates, reviews, ratings, publishers, and authors, to identify patterns and trends.

**Key Findings**

1. **Abundance of Contemporary Content**: The database contains over 800 books published after 2000, indicating a wealth of recent and contemporary content. This highlights the relevance of incorporating features and analysis tailored to modern titles, catering to the preferences of contemporary readers.

2. **Diverse User Preferences**: Books in the database exhibit varying numbers of reviews and average ratings, reflecting the diverse preferences of users. Higher reviews and ratings often correlate with more popular and well-received books. Understanding these preferences can aid in curating personalized recommendations.

3. **Publisher Insights**: Penguin Books emerged as the publisher with the most books over 50 pages, boasting 42 titles in the database. The presence of major publishers indicates a diverse and robust catalog of books, presenting ample opportunities to cater to different reading tastes.

4. **Top-rated Authors**: Renowned authors such as J.K. Rowling, Markus Zusak, and J.R.R. Tolkien received high average ratings, reaffirming the positive reception of established and popular authors. Collaborating with such authors or promoting their works could be a strategic move.

5. **Active Reviewer Community**: Users who review a substantial number of books tend to be more active and engaged within the community. These prolific reviewers have diverse reading interests, and their insights provide an excellent opportunity to enable discovery through personalized recommendations.

**Conclusions and Recommendations**

1. **Diverse Catalog for Targeting Various Tastes**: The extensive collection of classic and contemporary titles in the database presents an opportunity to create a book-focused product that caters to a wide range of reading tastes. By leveraging this variety, the product can attract a broader audience.

2. **Personalized Recommendations for User Engagement**: The engaged reviewer community and user ratings and reviews offer valuable data for generating personalized recommendations. Implementing advanced recommendation algorithms can enhance user engagement and satisfaction.

3. **Understanding Success Factors**: Analyzing average ratings and reviews can provide deeper insights into the elements that contribute to successful books and popular authors. This knowledge can guide marketing strategies and content acquisition decisions.

4. **Targeting Prolific Reviewers**: Highly engaged prolific reviewers represent a valuable target audience. The product could incorporate social features to further encourage their participation, turning them into advocates for the platform.

In conclusion, the SQL analysis of the book database provides a strong foundation for developing a book-focused product that aligns with real user needs and preferences. By leveraging the insights gained, the product can offer personalized experiences, promote engagement, and effectively cater to the diverse interests of the reading community. Continuous monitoring of user behavior and feedback will further refine the product's value proposition and ensure its success in the competitive book market.